<a href="https://colab.research.google.com/github/gabrielcessil/ComputationalGraph/blob/main/AutoDiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Overview**

### In modern numerical computing and machine learning, automatic differentiation (AutoDiff) plays a crucial role in efficiently computing gradients for optimization problems. This project implements a computational graph-based automatic differentiation system, where mathematical expressions are represented as directed graphs. Each node in the graph corresponds to an operation (e.g., addition, multiplication, sine, power), while edges capture dependencies between variables.

# **Project Features**

## This project provides:

### - A class-based implementation of a computational graph, allowing users to construct expressions dynamically.
### - Support for common mathematical operations (addition, multiplication, exponentiation, trigonometric functions).
### - Graph-based differentiation, enabling efficient computation of gradients via backpropagation.
### - A structured visualization of the computational graph, helping users understand function composition.



## **Unfornutelly, the current state of the code still do not support matrices.**
## **Feel free to add examples and use it as a didactic source**


In [491]:
import numpy as np

class AutoDiff_CompGraph:
  def __init__(self):
    self.expression = {}
    self.derivatives = {}

    self.var_counter = 0
    self.signal = []

  def forward_result(self): return self.signal[-1]
  def get_derivatives(self):
    output_node = next(reversed(self.derivatives))

    # Initialize adjoints for all nodes as 0
    adjoints = {node: 0 for node in self.derivatives}

    # Set the adjoint of the output node to 1
    adjoints[output_node] = 1

    # Process nodes in reverse order (starting from the output node)
    for main_key in reversed(self.derivatives):
        # Current adjoint for main_key
        main_adj = adjoints[main_key]

        # Propagate adjoint backward to each dependency (sub_key) based on partial derivative
        for sub_key, partial_deriv in self.derivatives[main_key].items():
            # Accumulate the adjoint for sub_key
            adjoints[sub_key] += main_adj * partial_deriv

    return adjoints



  def add_to_graph(self, result_var, param_list, name):

      self.expression[result_var["id"]] = {"name": name, "result": result_var, "params": []}
      for param in param_list:
        self.expression[result_var["id"]]["params"].append(param["id"])

  def create_variable(self, a):
      var_id = f"var{self.var_counter}"
      self.var_counter += 1
      self.signal.append(a)
      return {"value": a, "id": var_id}

  def verify_instance(self, a):
     if isinstance(a, dict) and "value" in a and "id" in a: return a #return if already a variable
     else: return self.create_variable(a)  # Otherwise, create a new variable

  def print_computation_graph_tree(self):
    """Prints the computation graph as a vertical tree structure with variable IDs."""

    def print_tree(node, prefix=""):
        """Recursive function to display the computation graph in a tree format with variable IDs."""
        details = self.expression[node]
        op_name = details["name"].upper()  # Make operation names uppercase for readability
        result_value = details["result"]["value"]
        result_id = details["result"]["id"]
        params = details["params"]

        # Print current operation with ID, name, and result value
        print(f"{prefix}🔗 {result_id}: {op_name}")

        # Print children (dependencies) with indentation
        for i, param in enumerate(params):
            connector = "└──" if i == len(params) - 1 else "├──"
            print_tree(param, prefix + "   " + connector)

    # Find the output node (last operation in expression)
    output_node = next(reversed(self.expression), None)

    if not output_node:
        print("Computation Graph is empty.")
        return

    print("\nComputation Graph Tree:\n")
    print_tree(output_node)


  # SUPPORTED OPERATIONS FOR EQUATION BUILDING:

  def scalar(self, a) :
    a = self.verify_instance(a)
    self.derivatives[a["id"]] = {}
    self.add_to_graph(a,[],"scal")

    return a

  def pow(self,a,b):
    a_ = self.verify_instance(a)
    b_ = self.verify_instance(b)
    result = a_["value"]**b_["value"]
    result = self.create_variable(result)
    self.add_to_graph(result,[a_,b_],"pow")

    der_a = b_["value"]*a_["value"]**(b_["value"]-1)
    der_b = a_["value"]**b_["value"]*np.log(a_["value"])
    self.derivatives[result["id"]] = {a_["id"]:der_a, b_["id"]:der_b}

    return result

  def multiply(self,a,b):
    a_ = self.verify_instance(a)
    b_ = self.verify_instance(b)
    result = a_["value"]*b_["value"]
    result = self.create_variable(result)
    self.add_to_graph(result,[a_,b_ ],"mult")

    der_a = b_["value"]
    der_b = a_["value"]
    self.derivatives[result["id"]] = {a_["id"]:der_a, b_["id"]:der_b}
    return result

  def divide(self,a,b):
    a_ = self.verify_instance(a)
    b_ = self.verify_instance(b)
    result = a_["value"]/b_["value"]
    result = self.create_variable(result)
    self.add_to_graph(result,[a_,b_],"div")

    der_a = 1/b_["value"]
    der_b = -a_["value"]/(b_["value"]**2)
    self.derivatives[result["id"]] = {a_["id"]:der_a, b_["id"]:der_b}

    return result

  def add(self,a,b):
    a_ = self.verify_instance(a)
    b_ = self.verify_instance(b)
    result = a_["value"]+b_["value"]
    result = self.create_variable(result)
    self.add_to_graph(result,[a_,b_],"add")

    der_a = 1
    der_b = 1
    self.derivatives[result["id"]] = {a_["id"]:der_a, b_["id"]:der_b}

    return result

  def sub(self,a,b):
    a_ = self.verify_instance(a)
    b_ = self.verify_instance(b)
    result = a_["value"]-b_["value"]
    result = self.create_variable(result)
    self.add_to_graph(result,[a_,b_],"sub")

    der_a = 1
    der_b = -1
    self.derivatives[result["id"]] = {a_["id"]:der_a, b_["id"]:der_b}

    return result

  def sin(self,a):
    a_ = self.verify_instance(a)
    result = np.sin(a_["value"])
    result = self.create_variable(result)
    self.add_to_graph(result,[a_],"sin")

    der_a = np.cos(a_["value"])
    self.derivatives[result["id"]] = {a_["id"]:der_a}

    return result

  def cos(self,a):
    a_ = self.verify_instance(a)
    result = np.cos(a_["value"])
    result = self.create_variable(result)
    self.add_to_graph(result,[a_],"cos")

    der_a = - np.sin(a_["value"])
    self.derivatives[result["id"]] = {a_["id"]:der_a}

    return result


# **Example**

# Let's define the following example of $f(x,y,z)$

### $f(x,y,z) = (xy + y^{sin(z)})xy$

# Where the partial derivatives are analycally defined as:

### $\frac{df}{dx}=y(2xy+y^{sin(z)})$

### $\frac{df}{dy}=x(2xy+sin(z)y^{sin(z)}+y^{sin(z)})$

### $\frac{df}{dz}=xln(y)cos(z)y^{sin(z)+1}$

With these analytical forms we can verify the numerical results of the computational graph.

In [492]:
x = 1
y = 3
z = 0.5

f = (x*y + y**(np.sin(z)))*x*y

d_fx = y*(2*x*y+y**(np.sin(z)))

d_fy = x*(2*x*y+np.sin(z)*y**(np.sin(z))+y**(np.sin(z)))

d_fz = x*np.log(y)*np.cos(z)*y**(np.sin(z)+1)

In [493]:
def print_results(x, y, z, f, d_fx, d_fy, d_fz):
    """Prints the function and its partial derivatives in a clear format."""

    print("Function and Partial Derivatives:")
    print("-" * 40)  # Separator line

    print(f"Given:")
    print(f"  x = {x}")
    print(f"  y = {y}")
    print(f"  z = {z}")
    print("-" * 40)

    print(f"Function: f = (x*y + y**(sin(z)))*x*y") # Use sin() for clarity
    print(f"          f = {f}")  # Print the calculated value of f
    print("-" * 40)

    print("Partial Derivatives:")
    print(f"  df/dx = y*(2*x*y + y**(sin(z)))") # Use sin()
    print(f"        = {d_fx}") # Print the value
    print(f"  df/dy = x*(2*x*y + sin(z)*y**(sin(z)) + y**(sin(z)))") # Use sin()
    print(f"        = {d_fy}") # Print the value
    print(f"  df/dz = x*log(y)*cos(z)*y**(sin(z)+1)") # Use sin(), log(), cos()
    print(f"        = {d_fz}") # Print the value
    print("-" * 40)


In [494]:
print_results(x, y, z, f, d_fx, d_fy, d_fz)

Function and Partial Derivatives:
----------------------------------------
Given:
  x = 1
  y = 3
  z = 0.5
----------------------------------------
Function: f = (x*y + y**(sin(z)))*x*y
          f = 14.080019381708262
----------------------------------------
Partial Derivatives:
  df/dx = y*(2*x*y + y**(sin(z)))
        = 23.08001938170826
  df/dy = x*(2*x*y + sin(z)*y**(sin(z)) + y**(sin(z)))
        = 8.505170136634511
  df/dz = x*log(y)*cos(z)*y**(sin(z)+1)
        = 4.897763459363576
----------------------------------------


# **Using the computational graph**

### Here we rebuild $f(x,y,z)$ using the computational graph object:

In [495]:
# Arbitraty Example with the computational graph
def Example_FUNC(x, y, z):
    # COmputation Graph object
    ad = AutoDiff_CompGraph()

    # Scalars with the given valuesreturn a
    var_x = ad.scalar(x)
    print("x is assigned to ", var_x["id"], "\n")
    var_y = ad.scalar(y)
    print("y is assigned to ", var_y["id"], "\n")
    var_z = ad.scalar(z)
    print("z is assigned to ", var_z["id"], "\n")

    # Build the equation sequentially
    op1 = ad.multiply(var_x,var_y)
    print("Op1 is assinged to ", op1["id"], "\n")

    op2 = ad.sin(var_z)
    print("Op2 is assinged to ", op2["id"], "\n")

    op3 = ad.pow(var_y,op2)
    print("Op3 is assinged to ", op3["id"], "\n")

    op4 = ad.add(op1, op3)
    print("Op4 is assinged to ", op4["id"], "\n")

    result = ad.multiply(op4, op1)
    print("Result is assinged to ", result["id"], "\n\n")

    return ad

In [496]:
# Assign numeric values to the example function
compGrap = Example_FUNC(x,y,z)

x is assigned to  var0 

y is assigned to  var1 

z is assigned to  var2 

Op1 is assinged to  var3 

Op2 is assinged to  var4 

Op3 is assinged to  var5 

Op4 is assinged to  var6 

Result is assinged to  var7 




In [497]:
# Get the derivatives,
derivatives = compGrap.get_derivatives()

In [498]:
f = compGrap.forward_result()
auto_dfx = derivatives["var0"]
auto_dfy = derivatives["var1"]
auto_dfz = derivatives["var2"]
print_results(x, y, z, f, auto_dfx, auto_dfy, auto_dfz)

Function and Partial Derivatives:
----------------------------------------
Given:
  x = 1
  y = 3
  z = 0.5
----------------------------------------
Function: f = (x*y + y**(sin(z)))*x*y
          f = 14.080019381708262
----------------------------------------
Partial Derivatives:
  df/dx = y*(2*x*y + y**(sin(z)))
        = 23.08001938170826
  df/dy = x*(2*x*y + sin(z)*y**(sin(z)) + y**(sin(z)))
        = 8.505170136634511
  df/dz = x*log(y)*cos(z)*y**(sin(z)+1)
        = 4.897763459363575
----------------------------------------


In [499]:
compGrap.print_computation_graph_tree()


Computation Graph Tree:

🔗 var7: MULT
   ├──🔗 var6: ADD
   ├──   ├──🔗 var3: MULT
   ├──   ├──   ├──🔗 var0: SCAL
   ├──   ├──   └──🔗 var1: SCAL
   ├──   └──🔗 var5: POW
   ├──   └──   ├──🔗 var1: SCAL
   ├──   └──   └──🔗 var4: SIN
   ├──   └──   └──   └──🔗 var2: SCAL
   └──🔗 var3: MULT
   └──   ├──🔗 var0: SCAL
   └──   └──🔗 var1: SCAL
